### 0.1 Setup

In [1]:
from pyspark.sql import SparkSession

#Create SparkSession
spark = SparkSession.builder.appName('govtech_challenge').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/26 11:05:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [3]:
spark.sparkContext.getConf().getAll()

[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'govtech_challenge'),
 ('spark.app.startTime', '1669431910567'),
 ('spark.app.id', 'local-1669431911555'),
 ('spark.driver.port', '54685'),
 ('spark.rdd.compress', 'True'),
 ('spa

### 0.2 Task

Application mobile number is 8 digits Applicant is over 18 years old as of 1 Jan 2022 Applicant has a valid email (email ends with @emailprovider.com or @emailprovider.net) You are required to format datasets in the following manner:

Split name into first_name and last_name Format birthday field into YYYYMMDD Remove any rows which do not have a name field (treat this as unsuccessful applications) Create a new field named above_18 based on the applicant's birthday Membership IDs for successful applications should be the user's last name, followed by a SHA256 hash of the applicant's birthday, truncated to first 5 digits of hash (i.e <last_name>_<hash(YYYYMMDD)>) You are required to consolidate these datasets and output the successful applications into a folder, which will be picked up by downstream engineers. Unsuccessful applications should be condolidated and dropped into a separate folder.

You can use common scheduling solutions such as cron or airflow to implement the scheduling component. Please provide a markdown file as documentation.

### 0.3 Read

In [37]:
import os
upstream_dir = "/Users/tonyngmk/repo/Data-Engineer-Tech-Challenge/"
upstream_csv = [f for f in os.listdir(upstream_dir) if f.endswith(".csv")]
upstream_csv

['applications_dataset_1.csv', 'applications_dataset_2.csv']

In [39]:
sdf = spark.read.format("csv").load(upstream_dir, header=True) # read entire directory
# problematic if directory is used for multiple input source of dfiferent schema

In [43]:
sdf.printSchema()

row = sdf.count()
col = len(sdf.columns)
print(f"Size: {row}*{col}") 

root
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- mobile_no: string (nullable = true)

Size: 5072*4


In [41]:
sdf.limit(5).show(truncate=False) # Peek

+---------------+-----------------------------+-------------+---------+
|name           |email                        |date_of_birth|mobile_no|
+---------------+-----------------------------+-------------+---------+
|Tony Shepherd  |Tony_Shepherd@petersen.com   |07/03/2016   |711447   |
|Sherry Gonzalez|Sherry_Gonzalez@caldwell.biz |14-03-1973   |66744895 |
|Ashlee Austin  |Ashlee_Austin@melendez.com   |12/09/1992   |6454197  |
|David Brown    |David_Brown@jackson-smith.biz|2001-09-22   |69082983 |
|Marc Meyer     |Marc_Meyer@chavez.com        |1996/05/17   |9727376  |
+---------------+-----------------------------+-------------+---------+



In [47]:
# Final function
def read(upstream_dir):
    sdf = spark.read.format("csv").load(upstream_dir, header=True)
    return sdf

sdf = read("/Users/tonyngmk/repo/Data-Engineer-Tech-Challenge/")

### 1. Check mobile number
- 8 digits

In [48]:
# sdf.filter(len(sdf.mobile_no) != 8)

TypeError: object of type 'Column' has no len()